# Simple Example: Stable Dynamics Moldel and Beckmann Model

In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [2]:
import numba
numba.__version__

'0.51.0'

In [3]:
sd_save = beckmann_save = './'
net_name = 'Example_net.tntp'
trips_name = 'Example_trips.tntp'

![title](github_example_pic.png)

- You can find the example's description at the following link:

https://doi.org/10.3390/math9111217

- There were more cases discussed in the following paper:

Nesterov, Y., de Palma, A. Stationary Dynamic Solutions in Congested Transportation Networks: Summary and Perspectives. Networks and Spatial Economics 3, 371–395 (2003). https://doi.org/10.1023/A:1025350419398

# Beckmann model

In [4]:
handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

model = md.Model(graph_data, graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,1.0
1,1,True,2,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0


In [5]:
graph_correspondences

{1: {'targets': [1, 2], 'corrs': [0.0, 2000.0]},
 2: {'targets': [2, 1], 'corrs': [0.0, 3000.0]}}

## Frank-Wolfe method

In [6]:
assert(model.mu == 0.25)
max_iter = 1000

print('Frank-Wolfe without stopping criteria')
solver_kwargs = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : True, 'verbose_step': 200, 'save_history' : True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'fwm', solver_kwargs = solver_kwargs)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

Frank-Wolfe without stopping criteria
Oracles created...
Frank-Wolfe method...
Primal_init = 2757.81
Dual_init = -2500
Duality_gap_init = 257.812

Iterations number: 200
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = 0
Duality_gap / Duality_gap_init = 0

Iterations number: 400
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = 3.18323e-12
Duality_gap / Duality_gap_init = 1.23471e-14

Iterations number: 600
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = 3.18323e-12
Duality_gap / Duality_gap_init = 1.23471e-14

Iterations number: 800
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = -1.36424e-12
Duality_gap / Duality_gap_init = -5.29161e-15

Iterations number: 1000
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = -4.54747e-13
Duality_gap / Duality_gap_init = -1.76387e-15

Result: success
Total iters: 1000
Primal_func_value = 2757.81
Dual_func_value = -2757.81
Duality_gap = -4.54747e-13
Du

In [7]:
print("flows:", result['flows'])
print("times:", result['times'])

flows: [  -0. 2000. 3000.   -0.]
times: [1.        0.575     0.8796875 1.       ]


# Stable Dynamics Model

parameter $\mu = 0$

In [8]:
handler = dh.DataHandler()
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
init_capacities = np.copy(graph_data['graph_table']['capacity'])
graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,1.0
1,1,True,2,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0


In [9]:
graph_correspondences

{1: {'targets': [1, 2], 'corrs': [0.0, 2000.0]},
 2: {'targets': [2, 1], 'corrs': [0.0, 3000.0]}}

## Step 1: Base flows
First of all, we should find admissible set of flows on the transport graph. It is required for defining duality gap.

In [10]:
import numba

In [11]:
# start from 0.5,  0.75, 0.875 (according to our flows reconstruction method) 
alpha = 0.75
graph_data['graph_table']['capacity'] = init_capacities * alpha
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,1500.0,1.0
1,1,True,2,True,1500.0,0.5
2,2,True,1,True,1500.0,0.5
3,2,True,1,True,1500.0,1.0


In [12]:
assert(model.mu == 0)
max_iter = 1000

solver_kwargs = {'eps_abs': 100, 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 500, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                solver_kwargs = solver_kwargs,
                                base_flows = alpha * graph_data['graph_table']['capacity'])
                                #base_flows here doesn't define anything now
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic

base_flows = result['flows']

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 2500
Dual_init = -2500
Duality_gap_init = 700

Iterations number: 500
Inner iterations number: 1011
Primal_func_value = 3479.76
Dual_func_value = -3499.96
Duality_gap = -25.8347
Duality_gap / Duality_gap_init = -0.0369067

Iterations number: 1000
Inner iterations number: 2011
Primal_func_value = 3488.97
Dual_func_value = -3499.92
Duality_gap = -14.2657
Duality_gap / Duality_gap_init = -0.0203795

Result: success
Total iters: 1000
Primal_func_value = 3488.97
Dual_func_value = -3499.92
Duality_gap = -14.2657
Duality_gap / Duality_gap_init = -0.0203795
Oracle elapsed time: 9 sec
Elapsed time: 10 sec
Time ratio = 2.0000443910173455
Flow excess = 0.0075024949182886935



__Base flows found numerically should be nonnegative and meet capacity constraints.__

__During these iterations, duality gap metric is irrelevant.__ On the next step ("SD Model solution"), duality gap -- based on the found base flows -- will be nonnegative and decrease as the algorithm approaches the problem's solution.

In [13]:
print("base flows:", result['flows'])
print("base times:", result['times'])

base flows: [ 489.1953825  1510.8046175  1511.25374238 1488.74625762]
base times: [1.         0.99989837 1.0000222  1.        ]


In [14]:
#with open(sd_save + 'anaheim_' + 'ustm' + '_base_flows_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
#    pickle.dump(base_flows, f)

## Step 2: SD Model solution

In [15]:
graph_data['graph_table']['capacity'] = init_capacities
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,1.0
1,1,True,2,True,2000.0,0.5
2,2,True,1,True,2000.0,0.5
3,2,True,1,True,2000.0,1.0


## Universal Similar Triangles

In [16]:
assert(model.mu == 0)
max_iter = 1000

solver_kwargs = {'eps_abs': 100,
                 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 400, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                solver_kwargs = solver_kwargs, base_flows = base_flows)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')
# NOTE: duality gap should be nonnegative here!

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 2500
Dual_init = -2500
Duality_gap_init = 664.298

Iterations number: 400
Inner iterations number: 810
Primal_func_value = 2985.03
Dual_func_value = -2999.95
Duality_gap = 14.1678
Duality_gap / Duality_gap_init = 0.0213275

Iterations number: 800
Inner iterations number: 1611
Primal_func_value = 2993
Dual_func_value = -2999.97
Duality_gap = 6.84015
Duality_gap / Duality_gap_init = 0.0102968

Result: success
Total iters: 1000
Primal_func_value = 2993.68
Dual_func_value = -3000
Duality_gap = 6.16647
Duality_gap / Duality_gap_init = 0.00928268
Oracle elapsed time: 11 sec
Elapsed time: 11 sec
Time ratio = 2.0000042745029365
Flow excess = 0.006315644518835706



In [17]:
print("flows:", result['flows'])
print("times: ", result['times'])

flows: [  -0.         2000.         2012.63128904  987.36871096]
times:  [1.         0.5        1.00000214 1.        ]
